In [ ]:
#sialencen8.22
# -*- coding: utf-8 -*-
import urllib.request
import json
import requests
import os

path = 'D:\\python爬虫网上数据\\'#上级文件夹

id = '5204648222'#微博id号
proxy_addr = "122.241.72.191:808"
pic_num = 0
weibo_name = "me3"#自行修改名称，自动新建文件夹、当本身存在这个文件夹时不会存储图片
#例子存储文件夹地址：D:\python爬虫网上数据\绿洲摄影2


def use_proxy(url, proxy_addr):
    req = urllib.request.Request(url)
    req.add_header("User-Agent",
                   "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy = urllib.request.ProxyHandler({'http': proxy_addr})
    opener = urllib.request.build_opener(proxy, urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data = urllib.request.urlopen(req).read().decode('utf-8', 'ignore')
    return data


def get_containerid(url):
    data = use_proxy(url, proxy_addr)
    content = json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if (data.get('tab_type') == 'weibo'):
            containerid = data.get('containerid')
    return containerid


def get_userInfo(id):
    url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value=' + id
    data = use_proxy(url, proxy_addr)
    content = json.loads(data).get('data')
    profile_image_url = content.get('userInfo').get('profile_image_url')
    description = content.get('userInfo').get('description')
    profile_url = content.get('userInfo').get('profile_url')
    verified = content.get('userInfo').get('verified')
    guanzhu = content.get('userInfo').get('follow_count')
    name = content.get('userInfo').get('screen_name')
    fensi = content.get('userInfo').get('followers_count')
    gender = content.get('userInfo').get('gender')
    urank = content.get('userInfo').get('urank')
    print("微博昵称：" + name + "\n" + "微博主页地址：" + profile_url + "\n" + "微博头像地址：" + profile_image_url + "\n" + "是否认证：" + str(
        verified) + "\n" + "微博说明：" + description + "\n" + "关注人数：" + str(guanzhu) + "\n" + "粉丝数：" + str(
        fensi) + "\n" + "性别：" + gender + "\n" + "微博等级：" + str(urank) + "\n")


def get_weibo(id, file):
    global pic_num
    i =1 #_______【修改你想开始爬取的起始页】————————
    pagnum = i+5#————————【修改你想下载的页数】————————————
    while True:
        url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value=' + id
        weibo_url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value=' + id + '&containerid=' + get_containerid(
            url) + '&page=' + str(i)
        try:
            data = use_proxy(weibo_url, proxy_addr)
            content = json.loads(data).get('data')
            cards = content.get('cards')
            if (len(cards) > 0):
                for j in range(len(cards)):
                    print("-----正在爬取第" + str(i) + "页，第" + str(j) + "条微博------")
                    card_type = cards[j].get('card_type')
                    if (card_type == 9):
                        mblog = cards[j].get('mblog')
                        attitudes_count = mblog.get('attitudes_count')
                        comments_count = mblog.get('comments_count')
                        created_at = mblog.get('created_at')
                        reposts_count = mblog.get('reposts_count')
                        scheme = cards[j].get('scheme')
                        text = mblog.get('text')
                        if mblog.get('pics') != None:
                            # print(mblog.get('original_pic'))
                            # print(mblog.get('pics'))
                            pic_archive = mblog.get('pics')
                            for _ in range(len(pic_archive)):
                                pic_num += 1
                                print(pic_archive[_]['large']['url'])
                                imgurl = pic_archive[_]['large']['url']
                                img = requests.get(imgurl)
                                f = open(path + weibo_name + '\\' + str(pic_num) + str(imgurl[-4:]),
                                         'ab')  # 存储图片，多媒体文件需要参数b（二进制文件）
                                f.write(img.content)  # 多媒体存储content
                                f.close()

                        with open(file, 'a', encoding='utf-8') as fh:
                            fh.write("----第" + str(i) + "页，第" + str(j) + "条微博----" + "\n")
                            fh.write("微博地址：" + str(scheme) + "\n" + "发布时间：" + str(
                                created_at) + "\n" + "微博内容：" + text + "\n" + "点赞数：" + str(
                                attitudes_count) + "\n" + "评论数：" + str(comments_count) + "\n" + "转发数：" + str(
                                reposts_count) + "\n")
                i += 1
                #if i==pagnum:#[当从1页开启时全部，则删除这个if]
                    #break
            else:
                break
            
        except Exception as e:
            print(e)
            pass


if __name__ == "__main__":
    if os.path.isdir(path + weibo_name):
        pass
    else:
        os.mkdir(path + weibo_name)
    file = path + weibo_name + '\\' + weibo_name + ".txt"
    get_userInfo(id)
    get_weibo(id, file)


微博昵称：跟着小鹿笑一个吧
微博主页地址：https://m.weibo.cn/u/5204648222?uid=5204648222&luicode=10000011&lfid=1005055204648222
微博头像地址：https://tvax4.sinaimg.cn/crop.0.0.512.512.180/005GeaL4ly8gxl9jstlugj30e80e8js4.jpg?KID=imgbed,tva&Expires=1650218591&ssig=64s0KLToYQ
是否认证：False
微博说明：平安顺遂
关注人数：546
粉丝数：192
性别：m
微博等级：29

-----正在爬取第1页，第0条微博------
https://wx3.sinaimg.cn/large/005GeaL4gy1h18bhoj014j30u01t075c.jpg
https://wx1.sinaimg.cn/large/005GeaL4gy1h18bhrljrtj30u01uo4lc.jpg
https://wx4.sinaimg.cn/large/005GeaL4gy1h18bhsfx9vj31ex0u0ac5.jpg
https://wx2.sinaimg.cn/large/005GeaL4gy1h18bhxrwdaj31900u0tc3.jpg
-----正在爬取第1页，第1条微博------
-----正在爬取第1页，第2条微博------
-----正在爬取第1页，第3条微博------
-----正在爬取第1页，第4条微博------
https://wx3.sinaimg.cn/large/005GeaL4gy1h0z5ad4gi3j30xb0ju3z2.jpg
-----正在爬取第1页，第5条微博------
-----正在爬取第1页，第6条微博------
https://wx3.sinaimg.cn/large/005GeaL4gy1h0vrna4o8kj30u0190ql5.jpg
https://wx3.sinaimg.cn/large/005GeaL4gy1h0vrnbjh0vj30u0140gxg.jpg
https://wx1.sinaimg.cn/large/005GeaL4gy1h0vrnc4tb8j30u00gz0yd.jp

-----正在爬取第8页，第6条微博------
https://wx4.sinaimg.cn/large/005GeaL4gy1gx0td1dwq0j30u011eju1.jpg
-----正在爬取第8页，第7条微博------
-----正在爬取第8页，第8条微博------
https://wx3.sinaimg.cn/large/005GeaL4gy1gwxnbt086yj30u0140dj2.jpg
https://wx1.sinaimg.cn/large/005GeaL4gy1gwxnbtmkx6j30tz1hc40d.jpg
https://wx4.sinaimg.cn/large/005GeaL4gy1gwxnbu8vs7j30u01uo781.jpg
https://wx2.sinaimg.cn/large/005GeaL4gy1gwxnbuuwc6j30pb0p4taq.jpg
https://wx4.sinaimg.cn/large/005GeaL4gy1gwxnbvic61j30u014043i.jpg
https://wx1.sinaimg.cn/large/005GeaL4gy1gwxnbw22n0j30u01403zg.jpg
https://wx3.sinaimg.cn/large/005GeaL4gy1gwxnbwqmtlj30u01szq5u.jpg
https://wx2.sinaimg.cn/large/005GeaL4gy1gwxnbxtc24j318g0u0tey.jpg
https://wx1.sinaimg.cn/large/005GeaL4gy1gwxnbyeg9bj30u00u0wgq.jpg
-----正在爬取第8页，第9条微博------
https://wx4.sinaimg.cn/large/005GeaL4gy1gwrmnmebn0j30m80fot9k.jpg
-----正在爬取第9页，第0条微博------
-----正在爬取第9页，第1条微博------
-----正在爬取第9页，第2条微博------
-----正在爬取第9页，第3条微博------
-----正在爬取第9页，第4条微博------
https://wx4.sinaimg.cn/large/005GeaL4gy1gw5y86m7v

In [ ]:
Jupyter 快捷键总结
Jupyter Notebook 有两种键盘输入模式。

编辑模式，允许你往单元中键入代码或文本；这时的单元框线是绿色的。
命令模式，键盘输入运行程序命令；这时的单元框线是灰色。
命令模式 (按键 Esc 开启)快捷键：
Enter : 转入编辑模式
Shift-Enter : 运行本单元，选中下个单元
Ctrl-Enter : 运行本单元
Alt-Enter : 运行本单元，在其下插入新单元
Y : 单元转入代码状态
M :单元转入markdown状态
R : 单元转入raw状态
设定 1 级标题
设定 2 级标题
设定 3 级标题
设定 4 级标题
设定 5 级标题
设定 6 级标题
Up : 选中上方单元
K : 选中上方单元
Down : 选中下方单元
J : 选中下方单元
Shift-K : 扩大选中上方单元
Shift-J : 扩大选中下方单元
A : 在上方插入新单元
B : 在下方插入新单元
X : 剪切选中的单元
C : 复制选中的单元
Shift-V : 粘贴到上方单元
V : 粘贴到下方单元
Z : 恢复删除的最后一个单元
D,D : 删除选中的单元
Shift-M : 合并选中的单元
Ctrl-S : 文件存盘
S : 文件存盘
L : 转换行号
O : 转换输出
Shift-O : 转换输出滚动
Esc : 关闭页面
Q : 关闭页面
H : 显示快捷键帮助
I,I : 中断Notebook内核
0,0 : 重启Notebook内核
Shift : 忽略
Shift-Space : 向上滚动
Space : 向下滚动
编辑模式 ( Enter 键启动)下快捷键
Tab : 代码补全或缩进
Shift-Tab : 提示
Ctrl-] : 缩进
Ctrl-[ : 解除缩进
Ctrl-A : 全选
Ctrl-Z : 复原
Ctrl-Shift-Z : 再做
Ctrl-Y : 再做
Ctrl-Home : 跳到单元开头
Ctrl-Up : 跳到单元开头
Ctrl-End : 跳到单元末尾
Ctrl-Down : 跳到单元末尾
Ctrl-Left : 跳到左边一个字首
Ctrl-Right : 跳到右边一个字首
Ctrl-Backspace : 删除前面一个字
Ctrl-Delete : 删除后面一个字
Esc : 进入命令模式
Ctrl-M : 进入命令模式
Shift-Enter : 运行本单元，选中下一单元
Ctrl-Enter : 运行本单元
Alt-Enter : 运行本单元，在下面插入一单元
Ctrl-Shift-- : 分割单元
Ctrl-Shift-Subtract : 分割单元
Ctrl-S : 文件存盘
Shift : 忽略
Up : 光标上移或转入上一单元
Down :光标下移或转入下一单元
Ctrl+ /: 标记单元格内容并按Ctrl+ /.它将注释掉该单元格中的所有行.重复相同的步骤以取消注释单元格的行.